# CNE y CENSO 2022

Homologaremos las tablas de datos OCHA-ONU y CNE para poder cruzar toda la información. Haremos esto haciendo una unión por intersección de polígonos; sin embargo, al intersectar las geometrías obtendremos más de una coincidencia, por lo que será útil quedarnos con la mayor área. Además, para que tengamos un margen de error razonables, haremos que la coincidencia del área sea de al menos el 80%.

In [26]:
import geopandas as gpd

# Cargar ambos shapefiles
shapefile1 = "data/maps/CNE_parroquias_desde2013.shp"
shapefile2 = "data/ecu_adm_2024/ecu_adm_adm3_2024.shp"

gdf1 = gpd.read_file(shapefile1)
# seleccionar columnas relevantes
gdf1 = gdf1[['CODPRO', 'CODCAN', 'CODPAR', 'geometry']]
gdf2 = gpd.read_file(shapefile2)
gdf2 = gdf2[['ADM3_PCODE', 'geometry']]

# Asegurar mismo CRS
if gdf1.crs is None:
    gdf1 = gdf1.set_crs(epsg=4326)
if gdf2.crs is None:
    gdf2 = gdf2.set_crs(epsg=4326)

gdf2 = gdf2.to_crs(gdf1.crs)

# Reproyectar a UTM para cálculos de área
gdf1 = gdf1.to_crs(epsg=32717)
gdf2 = gdf2.to_crs(epsg=32717)

# Opción 1: Usar 'within' en lugar de 'intersects'
unida = gpd.sjoin(gdf1, gdf2, how='left', predicate='intersects')
#within

<strong>Un caso puntual, para entender la lógica. </strong> <br>
Pra empezar, solo obtengamos una tabla de la columna B con los índices.

In [27]:
# Crear una tabla para recuperar geometrías de B
B_geom = gdf2[['geometry']].copy()
B_geom.index.name = 'index_right'
B_geom.columns = ['geometry_right']


Obtengamos el área de un caso

In [28]:
unidaTest = unida[unida['ADM3_PCODE'] == 'EC170150'].copy()

In [29]:
# Obtener cada B para cada A y calcular área de intersección
AB = unidaTest.join(B_geom, on='index_right', how='left')
AB["inter_area"] = AB.geometry.intersection(AB["geometry_right"]).area

In [30]:
# Luego eligo un área de intersección que sea al menos el %80 del área del polígino de A
AB['testu80'] = AB["inter_area"] >= .8*AB.geometry.area

# Y eligo el polígono B con mayor área de intersección que cumpla la condición
AB[AB['testu80'] == True].sort_values(by='inter_area', ascending=False).\
   drop_duplicates(subset=['ADM3_PCODE']).reset_index(drop=True)

,CODPRO,CODCAN,CODPAR,geometry,index_right,ADM3_PCODE,geometry_right,inter_area,testu80
0,17,60,7160,"POLYGON ((778871.488 9994525.038, 778883.42 99...",791,EC170150,"POLYGON ((778871.488 9994525.038, 778883.42 99...",5.418117e+07,True


## Caso general
Obtener una lista de las parroquias que tuvieron más de una coincidencia (por solapado). (Con el fin de iterar solo para lo que sea necesario.)

In [31]:
# Ver dónde se intersectó más de uno
recuentos = unida.groupby('ADM3_PCODE').size().reset_index(name='counts').sort_values(by='counts', ascending=False)

# obtener por máximo solapamiento
lista = recuentos[recuentos['counts'] > 1]
lista = lista['ADM3_PCODE'].tolist()

In [32]:
# Insto la primera fila, para armar el DataFrame apilando cada fila y ver el flujo de proceso
# (avances, por si algo se cayera y poder enternder qué paso donde dejó de ejecutarse)

unidaTest = unida[unida['ADM3_PCODE'] == lista[1]].copy()

# Obtener cada B para cada A y calcular área de intersección
AB = unidaTest.join(B_geom, on='index_right', how='left')
AB["inter_area"] = AB.geometry.intersection(AB["geometry_right"]).area

# Luego eligo un área de intersección que sea al menos el %80 del área del polígino de A
AB['testu80'] = AB["inter_area"] >= .8*AB.geometry.area

# Y eligo el polígono B con mayor área de intersección que cumpla la condición
filas = AB[AB['testu80'] == True].sort_values(by='inter_area', ascending=False).\
   drop_duplicates(subset=['ADM3_PCODE']).reset_index(drop=True) # Esta línea es la misma que usamos en estudiando un caso

In [33]:
filas

,CODPRO,CODCAN,CODPAR,geometry,index_right,ADM3_PCODE,geometry_right,inter_area,testu80
0,9,390,6150,"MULTIPOLYGON (((610770.588 9780143.105, 610950...",390,EC090150,"POLYGON ((612992.859 9781354.17, 612927.413 97...",1.567763e+09,True


In [34]:
import pandas as pd

In [35]:
# Ir ejecutando uno por uno
for codigo in lista[1:]:
   
   print(codigo)
   casoParticular = unida[unida['ADM3_PCODE'] == codigo] 

   # Obtener cada B para cada A y calcular área de intersección
   AB = casoParticular.join(B_geom, on='index_right', how='left')
   AB["inter_area"] = AB.geometry.intersection(AB["geometry_right"]).area

   # Luego eligo un área de intersección que sea al menos el %80 del área del polígino de A
   AB['testu80'] = AB["inter_area"] >= .8*AB.geometry.area

   # Y eligo el polígono B con mayor área de intersección que cumpla la condición
   fila_temp = AB[AB['testu80'] == True].sort_values(by='inter_area', ascending=False).\
      drop_duplicates(subset=['ADM3_PCODE']).reset_index(drop=True) # Esta línea es la misma que usamos en estudiando un caso
    
   # reemplazar en el dataframe original
   filas = pd.concat([filas, fila_temp])

   # progreso
   print(f'   {lista.index(codigo)/len(lista) * 100} % completado')

EC090150
   0.09615384615384616 % completado
EC010150
   0.19230769230769232 % completado
EC130150
   0.2884615384615385 % completado
EC130950
   0.38461538461538464 % completado
EC180150
   0.4807692307692308 % completado
EC120550
   0.576923076923077 % completado
EC070150
   0.6730769230769231 % completado
EC050150
   0.7692307692307693 % completado
EC230150
   0.8653846153846154 % completado
EC140157
   0.9615384615384616 % completado
EC170166
   1.0576923076923077 % completado
EC060150
   1.153846153846154 % completado
EC091650
   1.25 % completado
EC110950
   1.3461538461538463 % completado
EC060251
   1.4423076923076923 % completado
EC130650
   1.5384615384615385 % completado
EC100450
   1.6346153846153848 % completado
EC110150
   1.7307692307692308 % completado
EC100150
   1.826923076923077 % completado
EC100352
   1.9230769230769231 % completado
EC090750
   2.019230769230769 % completado
EC060350
   2.1153846153846154 % completado
EC141150
   2.2115384615384617 % completado
EC1

In [36]:
# preparas diccionario simple en excel
diccionario = filas[['CODPRO',	'CODCAN', 'CODPAR', 'ADM3_PCODE']]
diccionario.to_excel("data/diccionario_parroquias.xlsx", index=False)